<a href="https://colab.research.google.com/github/theosanderson/Prioritisation/blob/main/Cherry_picking_optimisation_static.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model
from io import StringIO
import pandas as pd
from collections import defaultdict


In [ ]:
past_6_days_io = StringIO(
"""ltla,genomes
London,25
Birmingham,25
Cardiff,25
Kent,0
""")
past_6_days = pd.read_csv(past_6_days_io)


case_numbers_io = StringIO(
"""ltla,cases
London,25
Birmingham,5
Cardiff,25
Kent,25
""")
case_numbers = pd.read_csv(case_numbers_io)

box_manifest_io = StringIO(
"""box,plate,coord,ltla,priority
boxC5,plate33B,A5,Cardiff,0
boxC2,plate123A,A1,Cardiff,0
box1,plate123A,A2,London,0
box1,plate1413A,A3,London,0
box1,plate15B,A4,Kent,0
box2,plate26A,A1,London,0
box2b,plate25B,A1,London,0
box3,plate23C,A1,Kent,0
box3,plate22D,A1,Birmingham,1
""")
box_manifest = pd.read_csv(box_manifest_io)

In [ ]:
box_manifest

,box,plate,coord,ltla,priority
0,boxC5,plate33B,A5,Cardiff,0
1,boxC2,plate123A,A1,Cardiff,0
2,box1,plate123A,A2,London,0
3,box1,plate1413A,A3,London,0
4,box1,plate15B,A4,Kent,0
5,box2,plate26A,A1,London,0
6,box2b,plate25B,A1,London,0
7,box3,plate23C,A1,Kent,0
8,box3,plate22D,A1,Birmingham,1


## Constants

In [ ]:
seconds_per_cherrypick = 10
seconds_per_box_load = 60*20

total_time_available = 60*20*2 +45 # fake small value

ltla_loss_weighting = 2

individual_sample_weighting =1

priority_sample_weighting = 10

## Variable set up

In [ ]:
model = cp_model.CpModel()

sample_is_picked = {} # indexed by row index
priority_sample_is_picked = {} # indexed by row index
box_is_picked = {} # indexed by box name

sample_is_picked_by_ltla = defaultdict(list)


boxes = box_manifest.box.unique().tolist()
for box_name in boxes:
  box_is_picked[box_name] = model.NewBoolVar(f'box_{box_name}_is_picked')

for i,row in box_manifest.iterrows():
  sample_is_picked[i] = model.NewBoolVar(f'sample_{i}_is_picked')
  model.Add(box_is_picked[row.box] >= sample_is_picked[i])
  sample_is_picked_by_ltla[row.ltla].append(sample_is_picked[i])
  if row.priority:
    priority_sample_is_picked[i] = sample_is_picked[i]

total_boxes_picked = sum(box_is_picked.values())
total_samples_picked = sum(sample_is_picked.values())
total_priority_samples_picked = sum(priority_sample_is_picked.values())


total_time = total_boxes_picked* seconds_per_box_load + total_samples_picked*seconds_per_cherrypick

### Geographical desired variable set up

In [ ]:
total_by_ltla = {}
for key, value in sample_is_picked_by_ltla.items():
  total_by_ltla[key] = sum(value)

case_numbers['proportion'] = case_numbers['cases'] / case_numbers['cases'].sum()
case_number_proportions = dict(zip(case_numbers['ltla'], case_numbers['proportion']))

desired_numbers_for_eod_by_ltla = {}
projected_numbers_for_eod_by_ltla = {}

genomes_in_last_6_days_by_ltla = dict(zip(past_6_days['ltla'], past_6_days['genomes']))
total_in_past_6_days = past_6_days['genomes'].sum()

for ltla in case_number_proportions.keys():
  desired_numbers_for_eod_by_ltla[ltla] = int( (7/6) * total_in_past_6_days * case_number_proportions[ltla]) 
  # This assumes we expect to go at about the same rate as last 6 days - we could also manually specify the number we roughly expect to run today
  # Unfortunately trying to do this on the fly with a division of the number we expect to do doesn't seem to play well with the solver.


for ltla in case_number_proportions.keys():
  projected_numbers_for_eod_by_ltla[ltla] = genomes_in_last_6_days_by_ltla[ltla] + total_by_ltla[ltla]

## Constraints

In [ ]:
model.Add(total_time<total_time_available)

## Losses

In [ ]:
ltla_losses = {}
for ltla in projected_numbers_for_eod_by_ltla.keys():
  possible_loss =  desired_numbers_for_eod_by_ltla[ltla] - projected_numbers_for_eod_by_ltla[ltla]
  ltla_loss_var = model.NewIntVar(0,100000000,f"positive_loss_for_{ltla}")
  model.Add(ltla_loss_var>=possible_loss) #this is how we implement a 0-cut off

  ltla_losses[ltla] = ltla_loss_var

ltla_loss = sum(ltla_losses.values())

loss = ltla_loss * ltla_loss_weighting - total_samples_picked*individual_sample_weighting -total_priority_samples_picked*priority_sample_weighting

# Run solver

In [ ]:

model.Minimize(loss)
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
status = solver.Solve(model)

In [ ]:
if status == cp_model.OPTIMAL:
  print(f"Optimal solution found, loss: {solver.Value(loss)}")
elif status== cp_model.FEASIBLE:
  print(f"Feasible solution found, with total loss {solver.Value(loss)}")
else:
  print(status)

Optimal solution found, loss: 42


In [ ]:
box_manifest['to_pick'] = [solver.Value(sample_is_picked[i]) for i in box_manifest.index]

box_manifest

,box,plate,coord,ltla,priority,to_pick
0,boxC5,plate33B,A5,Cardiff,0,0
1,boxC2,plate123A,A1,Cardiff,0,0
2,box1,plate123A,A2,London,0,1
3,box1,plate1413A,A3,London,0,0
4,box1,plate15B,A4,Kent,0,1
5,box2,plate26A,A1,London,0,0
6,box2b,plate25B,A1,London,0,0
7,box3,plate23C,A1,Kent,0,1
8,box3,plate22D,A1,Birmingham,1,1


## Debugging

In [ ]:
def print_dict(the_dict):
  print({i:solver.Value(x) for i,x in the_dict.items()})
print_dict(desired_numbers_for_eod_by_ltla)

{'London': 27, 'Birmingham': 5, 'Cardiff': 27, 'Kent': 27}


In [ ]:
print_dict(projected_numbers_for_eod_by_ltla)

{'London': 26, 'Birmingham': 26, 'Cardiff': 25, 'Kent': 2}


In [ ]:
print_dict(ltla_losses)

{'London': 1, 'Birmingham': 0, 'Cardiff': 2, 'Kent': 25}
